# 💰 DDL: Tabela Silver - `tb_custo_chamado`

**Origem:** Tabela Bronze `custos`
**Domínio:** Financeiro

## ⚙️ Decisões de Governança e Tipagem

### 1. Correção de Tipagem Financeira (Crítico)
Uma das transformações mais importantes da Silver ocorre aqui:
- **`vl_custo`:** Na Bronze, este campo chegava como `TEXT` (string). Na Silver, forçamos a conversão para **`DECIMAL(12, 10)`**.
  - *Por que Decimal?* Tipos `FLOAT` ou `DOUBLE` podem introduzir erros de arredondamento em operações financeiras. `DECIMAL` garante precisão exata.

### 2. Mapeamento de Schema (Bronze → Silver)
| Coluna Bronze | Coluna Silver | Tipo Novo | Justificativa |
|---|---|---|---|
| `id_custo` | `cd_custo` | `BIGINT` | PK. |
| `id_chamado` | `cd_chamado` | `BIGINT` | FK para `tb_chamado`. |
| `custo` | `vl_custo` | `DECIMAL(12, 10)` | Padronização de valor (`vl_`) e tipo monetário. |

### 3. Constraints
- **Primary Key:** `pk_custo` (`cd_custo`).
- **Relação:** 1:1 ou 1:N com `tb_chamado`.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_custo_chamado"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_custo BIGINT COMMENT 'Identificador do registro de custo.',
    cd_chamado BIGINT COMMENT 'Vínculo com o chamado.',
    vl_custo DECIMAL(12, 10) COMMENT 'Valor monetário do custo.',
    dt_ingestion TIMESTAMP COMMENT 'Data e hora da ingestão do registro na tabela.',
    dc_origem STRING COMMENT 'Identificação do sistema ou arquivo fonte dos dados.',
    CONSTRAINT pk_custo PRIMARY KEY (cd_custo)
)
COMMENT 'Tabela silver de custos associados a cada chamado';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")